In [147]:
import sys
import json
default = {'scenario': 'base', 'training_folder': '../..'} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))

sys.path.insert(0, r'../../quetzal') # Add path to quetzal
from quetzal.io import excel
import pandas as pd
import numpy as np

var = excel.read_var(file = '../inputs/parameters.xlsx', scenario = argv['scenario'])

In [39]:
file = '../inputs/parameters.xlsx'
default_scenario = 'base'
period = None

In [98]:
var = excel.read_var(file = '../inputs/parameters.xlsx', scenario = argv['scenario']).drop(('general', 'parent'))


var = var.to_frame().reset_index().set_index('category')
var = var.rename(columns={'parameter': 'text', default_scenario: 'value'})
var['name'] = var.index + '_' + var['text']


def records(df):
  if type(df) == pd.core.frame.DataFrame:
    return df.to_dict('records')
  elif type(df) == pd.core.frame.Series:
    return df.to_dict()

[{'category': cat, 'param': records(var.loc[cat])} for cat in var.index.unique()]

[{'category': 'preparation_footpaths',
  'param': [{'text': 'max_length',
    'value': 3000.0,
    'name': 'preparation_footpaths_max_length'},
   {'text': 'speed', 'value': 3.0, 'name': 'preparation_footpaths_speed'},
   {'text': 'n_clusters',
    'value': 3000,
    'name': 'preparation_footpaths_n_clusters'}]},
 {'category': 'preparation_ntlegs',
  'param': [{'text': 'n_ntlegs',
    'value': 5,
    'name': 'preparation_ntlegs_n_ntlegs'},
   {'text': 'max_ntleg_length',
    'value': 5000.0,
    'name': 'preparation_ntlegs_max_ntleg_length'},
   {'text': 'long_leg_speed',
    'value': 7.0,
    'name': 'preparation_ntlegs_long_leg_speed'},
   {'text': 'short_leg_speed',
    'value': 3.0,
    'name': 'preparation_ntlegs_short_leg_speed'},
   {'text': 'threshold',
    'value': 500.0,
    'name': 'preparation_ntlegs_threshold'}]},
 {'category': 'preparation_logit',
  'param': [{'text': 'time',
    'value': -0.00055,
    'name': 'preparation_logit_time'},
   {'text': 'price', 'value': -1.0,

In [162]:
var = excel.read_var(file = '../inputs/parameters.xlsx', scenario = argv['scenario']).drop(('general', 'parent')).to_frame()
parameter_frame = pd.read_excel(file, sheet_name='parameters').dropna(axis=1, how='all')

# types
try:
    types = parameter_frame.set_index(
        ['category', 'parameter']
    )['type'].dropna()
    js_types_dict = {'float': 'Number', 'int': 'Number', 'bool': 'Boolean', 'str': 'String'}
    var = var.join(types)
    var['type'] = var['type'].apply(lambda x: js_types_dict.get(x, x))
except KeyError:
    var['type'] = np.nan

# units
try:
    units = parameter_frame.set_index(
        ['category', 'parameter']
    )['unit'].dropna()
    var = var.join(units)
except KeyError:
    var['unit'] = np.nan

# hints
try:
    hints = parameter_frame.set_index(
        ['category', 'parameter']
    )['description'].dropna()
    var = var.join(hints)
except KeyError:
    var['description'] = np.nan

# rules
try:
    rules = parameter_frame.set_index(
        ['category', 'parameter']
    )['rules'].dropna()
    var = var.join(rules)
except KeyError:
    var['rules'] = np.full((len(var), 1), ['required']).tolist()

var = var.reset_index().set_index('category')
var = var.rename(columns={'parameter': 'text', default_scenario: 'value', 'unit': 'units', 'description': 'hint'})
var['name'] = var.index + '_' + var['text']
var = var.where(pd.notnull(var), None)


def records(df):
  if type(df) == pd.core.frame.DataFrame:
    return df.to_dict('records')
  elif type(df) == pd.core.frame.Series:
    return [df.to_dict()]

js_params = [{'category': cat, 'params': records(var.loc[cat])} for cat in var.index.unique()]

In [163]:
with open('params.json', 'w', encoding='utf-8-sig') as f:
    json.dump(js_params, f, ensure_ascii=False, indent=2)